# Correlation between mAP and Structural metrics

In [34]:
import json
import polars as pl
import numpy as np
from matplotlib import pyplot as plt
import os
import glob
from scipy.stats import pearsonr, t
from collections import defaultdict
import pingouin as pg

In [4]:
metrics = {}

In [5]:
with open("/home/upal/Data/ontologies/wines-ontology/data/term_typing/ranked_retrieval/0_shot/2024-09-05/dataset_metrics.json", "r") as f:
    metrics["wines-ontology"] = json.load(f)

In [6]:
with open("/home/upal/Data/ontologies/case-uco-owl-trafficking/data/term_typing/ranked_retrieval/0_shot/2024-09-05/dataset_metrics.json", "r") as f:
    metrics["case-uco-owl-trafficking"] = json.load(f)


In [7]:
with open("/home/upal/Data/ontologies/astronomy-ontology/data/term_typing/ranked_retrieval/0_shot/2024-11-04/dataset_metrics.json", "r") as f:
    metrics["astronomy-ontology"] = json.load(f)

In [8]:
pred_metrics = []

In [17]:
corr = defaultdict(list)

In [58]:
metric = "ontology_depth"

In [24]:
base_path = "/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval"

In [25]:
def correct_retrievability(preds, labels, l):
    ret = 0
    c_ret = 0
    for pred, label in zip(preds, labels):
        if l not in label:
            continue
        ret +=1
        #c_ret += l in pred
        if label.index(l) < len(pred):
            c_ret += int(l == pred[label.index(l)]) if l in label else 0.0
    return 1. * c_ret / ret if ret !=0 else 0.0

In [59]:
corr = defaultdict(list)
for fname in glob.iglob("/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval/**/predictions.json", recursive=True):
    if "run_1/" not in fname or "low" in fname or "high" in fname:
        print(fname)
        continue
    preds = pl.read_ndjson(fname)
    with open(os.path.join(os.path.dirname(fname), "pred_metrics.json"), "r") as f:
        pred_metrics = json.load(f)
    for ont in ["case-uco-owl-trafficking", "wines-ontology", "astronomy-ontology"]:
        for n_shot in [f"{i}_shot" for i in range (10)]:
            if ont in fname and n_shot in fname:
                label_path = [a for a in glob.iglob(f"/home/upal/PhD/Experiments/llm_ontology_awareness/data/**/term_typing_ranked_retrieval_dataset.json",
                                                    recursive=True) if ont in a and n_shot in a][0]
                with open(label_path, "r") as f:
                    labels = pl.read_ndjson(f)
                c_ret = {k: correct_retrievability(preds["Prediction"].to_list(), labels["Ranked List"].to_list(), k) for k in metrics[ont]["class_counts"].keys()}
                metric_score = [metrics[ont][metric][c] for c in c_ret.keys()]
                # df = pl.DataFrame({"C Ret": list(c_ret.values()), "Metric": metric_score, "Subject": list(c_ret.keys())}).to_pandas()
                corr[ont].append(pearsonr(list(c_ret.values()), metric_score))

/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval/8_shot/most_common/llama3-7B/wines-ontology/62951f46-95d6-46b2-afca-a04624186e44/run_2/predictions.json
/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval/8_shot/most_common/llama3-7B/wines-ontology/bb9cb7f3-f231-4c53-9d8f-871c712a13fa/run_2/predictions.json
/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval/8_shot/most_common/llama3-7B/wines-ontology/3d236c1d-6031-4dea-8b6e-f5e06ed9ff6a/run_2/predictions.json
/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval/8_shot/most_common/llama3-7B/wines-ontology/f684c22d-df21-4cbe-ab09-cbd68cb26bd7/run_2/predictions.json
/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval/8_shot/most_common/llama3-7B/astronomy-ontology/73697d75-aacd-4a68-b07d-547ee135f8ed/run_2/predictions.json
/home/upal/Results/llm_ontology_awareness/term_typing/ranked_retrieval/8_shot/most_common/llama3-7B/astronomy-ontology/f6

In [ ]:
len(corr)

In [33]:
corr

defaultdict(list,
            {'wines-ontology': [PearsonRResult(statistic=np.float64(-0.2700267312052242), pvalue=np.float64(0.06343478130111649)),
              PearsonRResult(statistic=np.float64(-0.3043721576411702), pvalue=np.float64(0.03543460297698181)),
              PearsonRResult(statistic=np.float64(-0.32082740689281236), pvalue=np.float64(0.026195408900403155)),
              PearsonRResult(statistic=np.float64(-0.3153448279222195), pvalue=np.float64(0.0290193524151762)),
              PearsonRResult(statistic=np.float64(-0.35859954974394825), pvalue=np.float64(0.012320878684590104)),
              PearsonRResult(statistic=np.float64(-0.28119266081826194), pvalue=np.float64(0.05285913799443779)),
              PearsonRResult(statistic=np.float64(-0.25271674412553835), pvalue=np.float64(0.08310032247358798)),
              PearsonRResult(statistic=np.float64(-0.2292617913064203), pvalue=np.float64(0.11700673778204393)),
              PearsonRResult(statistic=np.float64(-0.43

In [60]:
for ont, d in corr.items():
    avg_z_stat = np.mean(np.arctanh([v[0] for v in d]))
    r = np.tanh(avg_z_stat)
    t_stat = r * np.sqrt((len(d) - 2)) / (1 - np.pow(r, 2))
    p = 2 * min(t.cdf(t_stat, len(d) - 2), 1 - t.cdf(t_stat, len(d) - 2))
    print(ont, len(d), r, p)

wines-ontology 96 -0.24133287586050653 0.014742130547317633
astronomy-ontology 96 0.031083050854351455 0.7635830910565933
case-uco-owl-trafficking 96 0.1127745068818281 0.2709147629436468
